In [ ]:
#importar librerias
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import pandas as pd

In [ ]:
df_500 = pd.read_csv(r'./database/SP500_data_.csv', encoding = 'ISO-8859-1', delimiter = ',')

In [ ]:
df_500.columns = ['fecha', 'open', 'high', 'low', 'close', 'adj_close', 'volumen']

In [ ]:
df_500.head(3)

In [ ]:
df_500.columns

In [ ]:
#crear el objeto dash
app = dash.Dash()

In [ ]:
# DEFINICIÓN LAYOUT
app.layout = html.Div([
  html.Div([
    html.Label('Selección'),
    dcc.Dropdown(
      id = 'select',
      options = [
        {
          'label':'Apertura',
          'value':'open'
        },
        {
          'label':'Cierre',
          'value':'close'
        }
      ],
      value = 'close'
    )
  ],
    style = {
      'width':'48%',
      'display':'inline-block'
    }
  ),
  html.Div([
    html.Label('Rango Fechas'),
    dcc.DatePickerRange(
    id = 'select_date',
    initial_visible_month = df_500['fecha'].min(),
    start_date = df_500['fecha'].min(),
    end_date = df_500['fecha'].max()
  ) 
  ],
    style = {
      'width':'48%',
      'float':'right',
      'display': 'inline-block',
      'margin' :{'l': 0, 'b': 40, 't': 0, 'r': 10}
    }
  ),
  dcc.Graph(
    id = 'line_plot'
  ),
  dcc.Graph(
    id = 'bar_plot'
  )
],
  style = {
    'background': "#262626",
    'fontFamily': "Verdana",
    'color':'blue'
  }
)

In [ ]:
# CREACIÓN DE GRÁFICOS E INTERACTIVIDAD
#Callback para actualizar gráfico de cotización en función del dropdown eligiendo apertura o cierre de sesión y según selector de fechas
@app.callback(
  Output('line_plot', 'figure'),
  [
    Input('select_date','start_date'),
    Input('select_date','end_date'),
    Input('select','value')
  ]
)
def dinamic_graph_line(fmin, fmax, select):
  filtro = df_500[(df_500['fecha']>= fmin) & (df_500['fecha']<= fmax)]
  if select =='open':
    return {
      'data':[
        go.Scatter(
          x = filtro['fecha'],
          y = filtro['open'],
          mode = 'lines'
        )
      ],
      'layout': go.Layout(
        title = 'SP500 Cotización',
        xaxis = {'title': 'Fecha'},
        yaxis = {'title': 'Valor de Cotización y Apertura'},
        hovermode = 'closest'
      )
    }
  else:
    return {
      'data':[
        go.Scatter(
          x = filtro['fecha'],
          y = filtro['close'],
          mode = 'lines'
        )
      ],
      'layout': go.Layout(
        title = 'SP500 Cotización',
        xaxis = {'title': 'Fecha'},
        yaxis = {'title': 'Valor de Cotización y Cierre'},
        hovermode = 'closest'
      )
    }

In [ ]:
#Callback para actualizar gráfico de volumen según selector de fechas
@app.callback(
  Output('bar_plot', 'figure'),
  [
    Input('select_date','start_date'),
    Input('select_date','end_date')
  ]
)
def dinamic_graph_bar(fmin, fmax):
  filtro = df_500[(df_500['fecha']>= fmin) & (df_500['fecha']<= fmax)]
  return{
    'data':[
      go.Bar(
        x = filtro['fecha'],
        y = filtro['volumen'],
      )
    ],
    'layout': go.Layout(
      title = 'SP500 Volumen Negociado',
      xaxis = {'title':'fecha'},
      yaxis = {'title':'Volumen'}
    )
  }

In [ ]:
#Sentencias para abrir el servidor al ejecutar este script
if __name__ == "__main__":
  app.run_server(port = 7996)